# Purpose

### 2022-04-06
In this notebook we'll get the subreddits relevant to a country so that we can do QA (check ratings & subreddit relevance) for an FPR in one Feed.

---

TODO: Haven't included place logic (e.g., add direction to: city, state, country subreddits.)


### Updates
2022-02-23:
I'm going to automate more of the notebook process into functions:
- automate "optimal" minimum number of subs to minimize number of orphans
- move df_clean creation & column order to a function
- update order of columns for cluster dfs

2022-02-16: 
I [David] will update the QA sheet so that we have standardized columns/format. Otherwise it'll be more work for us to wait for country managers to format things and then standardize them after the fact



# Imports & notebook setup

In [1]:
%load_ext autoreload
%autoreload 2

# Register bigquery magic (only needed for laptop/local, not colab)
# %load_ext google.cloud.bigquery

In [2]:
# colab auth for BigQuery, google drive, & google sheets (gspread)
from google.colab import auth, files, drive
from google.auth import default
import sys  # need sys for mounting gdrive path

auth.authenticate_user()
print('Authenticated')

Authenticated


## Install custom library

### Append google drive path so we can install library from there

In [3]:
# Attach google drive & import my python utility functions
# if drive.mount() fails, you can also:
#   MANUALLY CLICK ON "Mount Drive"
import sys


g_drive_root = '/content/drive'

try:
    drive.mount(g_drive_root, force_remount=True)
    print('   Authenticated & mounted Google Drive')
    
except Exception as e:
    try:
        drive._mount(g_drive_root, force_remount=True)
        print('   Authenticated & mounted Google Drive')
    except Exception as e:
        print(e)
        raise Exception('You might need to manually mount google drive to colab')

l_paths_to_append = [
    f'{g_drive_root}/MyDrive/Colab Notebooks',

    # need to append the path to subclu so that colab can import things properly
    f'{g_drive_root}/MyDrive/Colab Notebooks/subreddit_clustering_i18n'
]
for path_ in l_paths_to_append:
    if path_ in sys.path:
        sys.path.remove(path_)
    print(f" Appending path: {path_}")
    sys.path.append(path_)

Mounted at /content/drive
   Authenticated & mounted Google Drive
 Appending path: /content/drive/MyDrive/Colab Notebooks
 Appending path: /content/drive/MyDrive/Colab Notebooks/subreddit_clustering_i18n


### Install library

In [4]:
# install subclu & libraries needed to read parquet files from GCS & spreadsheets
#  make sure to use the [colab] `extra` because it includes colab-specific libraries
module_path = f"{g_drive_root}/MyDrive/Colab Notebooks/subreddit_clustering_i18n/[colab]"

!pip install -e $"$module_path" --quiet

     |████████████████████████████████| 10.1 MB 23.6 MB/s 
     |████████████████████████████████| 14.2 MB 46.6 MB/s 
     |████████████████████████████████| 965 kB 40.2 MB/s 
     |████████████████████████████████| 144 kB 56.7 MB/s 
     |████████████████████████████████| 76 kB 3.9 MB/s 
     |████████████████████████████████| 285 kB 47.5 MB/s 
     |████████████████████████████████| 13.2 MB 53.8 MB/s 
     |████████████████████████████████| 79.9 MB 108 kB/s 
     |████████████████████████████████| 136 kB 65.7 MB/s 
     |████████████████████████████████| 715 kB 58.4 MB/s 
     |████████████████████████████████| 112 kB 75.0 MB/s 
     |████████████████████████████████| 74 kB 2.6 MB/s 
     |████████████████████████████████| 62 kB 669 kB/s 
     |████████████████████████████████| 146 kB 45.8 MB/s 
     |████████████████████████████████| 1.1 MB 36.0 MB/s 
     |████████████████████████████████| 181 kB 71.0 MB/s 
     |████████████████████████████████| 79 kB 7.7 MB/s 
     |█████████████

## Regular Imports

In [5]:
import os
from datetime import datetime

from google.cloud import bigquery

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib_venn import venn2_unweighted, venn3_unweighted
from tqdm import tqdm

# auth for google sheets
import gspread


creds_, _ = default()
gc = gspread.authorize(creds_)

# os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-science-prod-218515'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-prod-165221'

## Custom imports

In [6]:
# subclu imports
import subclu
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric, reorder_array,
)
from subclu.models.clustering_utils import (
    create_dynamic_clusters,
    convert_distance_or_ab_to_list_for_fpr,
    reshape_df_to_get_1_cluster_per_row,
    get_primary_topic_mix_cols,
    create_dynamic_clusters_clean,
)

from subclu.models.reshape_clusters_v041 import (
    keep_only_target_labels,
    get_table_for_optimal_dynamic_cluster_params,
    get_dynamic_cluster_summary,
    flag_mature_clusters_to_exclude_from_qa,
)


setup_logging()
notebook_display_config()
print_lib_versions([gspread, pd, np])

python		v 3.7.13
===
gspread		v: 4.0.1
pandas		v: 1.3.5
numpy		v: 1.21.5


# Checklist to re-run for a country:

- change country name for google sheet name
- change country initial in google sheet
    - add google sheet KEY (after created)
- change country name in geo-relevance query

## Create google sheet for country outputs


In [7]:
%%time

country_name_sheet_ = 'Netherlands'
target_abbrev_ = 'NL'
GSHEET_KEY = '1KGFY2T-M61j-l-jsy2e9dDjoXMLXELpX1g85T5qZ87o'
GSHEET_NAME = f'i18n {country_name_sheet_} subreddits and clusters - model v0.4.1'


d_wsh_names = {
    'qa_ready': {
        'name': 'subs_need_to_be_rated',
    },
    'clusters_t2t_list_raw': {
        'name': f'raw_clusters_list_{target_abbrev_}_{target_abbrev_}',
    },
    'clusters_t2t_fpr_after_qa': {
        'name': f'fpr_clusters_after_qa_{target_abbrev_}_{target_abbrev_}',
    },
    'sub_raw': {
        'name': 'raw_data_per_subreddit',
    },
    # 'clusters_t2t_fpr_raw': {
    #     'name': f'raw_clusters_fpr_{target_abbrev_}_{target_abbrev_}',
    # },
}

if GSHEET_KEY is not None:
    sh = gc.open_by_key(GSHEET_KEY)
    print(f"Opening google worksheet: {GSHEET_NAME} ...")
else:
    print(f"** Creating google worksheet: {GSHEET_NAME} ...")
    sh = gc.create(GSHEET_NAME)

# create worksheets:
for _, d_ in d_wsh_names.items():
    sh_name = d_['name']
    try:
        d_['worksheet'] = sh.worksheet(sh_name)
        print(f"  Opening tab/sheet: {sh_name} ...")
    except Exception as e:
        print(f"  ** Creating tab/sheet: {sh_name} ...")
        d_['worksheet'] = sh.add_worksheet(sh_name, rows=5, cols=5)

if GSHEET_KEY is None:
    print(f"\n*** New sheet ID (assign it to GSHEET_KEY variable): ***\n{sh.id}\n")

** Creating google worksheet: i18n Netherlands subreddits and clusters - model v0.4.1 ...
  ** Creating tab/sheet: subs_need_to_be_rated ...
  ** Creating tab/sheet: raw_clusters_list_NL_NL ...
  ** Creating tab/sheet: fpr_clusters_after_qa_NL_NL ...
  ** Creating tab/sheet: raw_data_per_subreddit ...

*** New sheet ID (assign it to GSHEET_KEY variable): ***
1KGFY2T-M61j-l-jsy2e9dDjoXMLXELpX1g85T5qZ87o

CPU times: user 99.9 ms, sys: 7.24 ms, total: 107 ms
Wall time: 2.41 s


# Load data from BigQuery

## Load subreddit geo-relevance & cultural relevance metadata

This data is already in bigQuery so read it straight from there. We'll use it to filter out geo-relevant (German) subs.

Also add the latest ratings so that we can filter based on those.

English-speaking countries don't have ambassador subs right now, so we should be able to create a standard template and replace the country name for these queries.

### SQL geo & cultural

In [8]:
%%time
%%bigquery df_geo_and_lang --project data-science-prod-218515 

-- Select geo+cultural subreddits for a target country
--  And add latest rating & over_18 flags to exclude X-rated & over_18
DECLARE TARGET_COUNTRY STRING DEFAULT 'Netherlands';
DECLARE MIN_COUNTRY_STANDARDIZED_RELEVANCE NUMERIC DEFAULT 3.0;


SELECT
    s.* EXCEPT(over_18, pt, verdict) 
    , nt.rating_name
    , nt.primary_topic
    , nt.rating_short
    , slo.over_18
    , slo.allow_discovery
    , CASE 
        WHEN(COALESCE(slo.over_18, 'f') = 't') THEN 'over_18_or_X_M_D_V'
        WHEN(COALESCE(nt.rating_short, '') IN ('X', 'M', 'D', 'V')) THEN 'over_18_or_X_M_D_V'
        ELSE 'unrated_or_E'
    END AS grouped_rating

FROM `reddit-employee-datasets.david_bermejo.subclu_v0041_subreddit_clusters_c_a` AS t
    -- Inner join b/c we only want to keep subs that are geo-relevant AND in topic model
    INNER JOIN (
        SELECT *
        FROM `reddit-employee-datasets.david_bermejo.subclu_subreddit_geo_score_standardized_20220212`
        WHERE country_name = TARGET_COUNTRY
    ) AS s
        ON t.subreddit_id = s.subreddit_id

    -- Add rating so we can get an estimate for how many we can actually use for recommendation
    LEFT JOIN (
        SELECT *
        FROM `data-prod-165221.ds_v2_postgres_tables.subreddit_lookup`
        -- Get latest partition
        WHERE dt = DATE(CURRENT_DATE() - 2)
    ) AS slo
    ON s.subreddit_id = slo.subreddit_id
    LEFT JOIN (
        SELECT * FROM `data-prod-165221.cnc.shredded_crowdsource_topic_and_rating`
        WHERE pt = DATE(CURRENT_DATE() - 2)
    ) AS nt
        ON s.subreddit_id = nt.subreddit_id

    -- Exclude popular US subreddits
    -- Can't query this table from local notebook because of errors getting google drive permissions. smh, excludefor now
    LEFT JOIN `reddit-employee-datasets.david_bermejo.subclu_subreddits_top_us_to_exclude_from_relevance` tus
        ON s.subreddit_name = LOWER(tus.subreddit_name)

WHERE 1=1
    AND s.subreddit_name != 'profile'
    AND COALESCE(s.type, '') = 'public'
    AND COALESCE(s.verdict, 'f') <> 'admin_removed'
    AND COALESCE(slo.over_18, 'f') = 'f'
    AND COALESCE(nt.rating_short, '') NOT IN ('X', 'D')

    AND(
        s.geo_relevance_default = TRUE
        OR s.relevance_percent_by_subreddit = TRUE
        OR s.e_users_percent_by_country_standardized >= MIN_COUNTRY_STANDARDIZED_RELEVANCE
    )
    AND country_name IN (
            TARGET_COUNTRY
        )

    AND (
         -- Exclude subs that are top in US but we want to exclude as culturally relevant
         --  For simplicity, let's go with the English exclusion (more relaxed) than the non-English one
         COALESCE(tus.english_exclude_from_relevance, '') <> 'exclude'
    )

ORDER BY e_users_percent_by_country_standardized DESC, users_l7 DESC, subreddit_name
;

CPU times: user 360 ms, sys: 26.1 ms, total: 386 ms
Wall time: 11.5 s


### Check df with geo + language information

In [9]:
print(df_geo_and_lang.shape)

(197, 26)


In [10]:
df_geo_and_lang.iloc[:4, :9]

,subreddit_id,subreddit_name,country_name,geo_relevance_default,b_users_percent_by_subreddit,e_users_percent_by_country_standardized,c_users_percent_by_country,d_users_percent_by_country_rank,relevance_percent_by_subreddit
0,t5_30hrx,thenetherlands,Netherlands,False,0.613013,10.658802,0.028566,2,True
1,t5_2r0ol,netherlands,Netherlands,False,0.398765,10.534847,0.018091,13,True
2,t5_2r117,amsterdam,Netherlands,False,0.640630,10.380873,0.009853,53,True
3,t5_39bxv,ik_ihe,Netherlands,False,0.570006,10.117907,0.010426,49,True


## Load model labels (clusters)

The clusters now live in a Big Query table and have standardized names, so pull the data from there.

### SQL labels


In [11]:
%%time
%%bigquery df_labels --project data-science-prod-218515 

-- select subreddit clusters from bigQuery

SELECT
    sc.subreddit_id
    , sc.subreddit_name
    , nt.primary_topic

    , sc.* EXCEPT(subreddit_id, subreddit_name, primary_topic_1214)
FROM `reddit-employee-datasets.david_bermejo.subclu_v0041_subreddit_clusters_c_a` sc
    LEFT JOIN (
        -- New view should be visible to all, but still comes from cnc_taxonomy_cassandra_sync
        SELECT * FROM `data-prod-165221.cnc.shredded_crowdsource_topic_and_rating`
        WHERE DATE(pt) = (CURRENT_DATE() - 2)
    ) AS nt
        ON sc.subreddit_id = nt.subreddit_id
;

CPU times: user 8.29 s, sys: 481 ms, total: 8.77 s
Wall time: 24.6 s


### Check label outputs

In [12]:
print(df_labels.shape)
df_labels.iloc[:4, :9]

(49558, 51)


,subreddit_id,subreddit_name,primary_topic,model_sort_order,posts_for_modeling_count,k_0013_label,k_0023_label,k_0041_label,k_0059_label
0,t5_5a9iie,progonlydj,None,40079,1000,12,19,34,49
1,t5_2x9c7,googleplaymusic,Music,40080,31,12,19,34,49
2,t5_3jzsk,ravedj,Music,40081,1000,12,19,34,49
3,t5_2rgie,happyhardcore,Music,40082,152,12,19,34,49


In [13]:
counts_describe(df_labels.iloc[:, :9])

,dtype,count,unique,unique-percent,null-count,null-percent
subreddit_id,object,"49,558","49,558",100.00%,0,0.00%
subreddit_name,object,"49,558","49,558",100.00%,0,0.00%
primary_topic,object,"41,366",52,0.13%,"8,192",16.53%
model_sort_order,int64,"49,558","49,558",100.00%,0,0.00%
posts_for_modeling_count,int64,"49,558",999,2.02%,0,0.00%
k_0013_label,int64,"49,558",13,0.03%,0,0.00%
k_0023_label,int64,"49,558",23,0.05%,0,0.00%
k_0041_label,int64,"49,558",41,0.08%,0,0.00%
k_0059_label,int64,"49,558",59,0.12%,0,0.00%


# Reshape data
Apply reshaping fxns so that we can export the data in a format that's good for QA.

## Keep only labels for Target subreddits


In [14]:
%%time
df_labels_target = keep_only_target_labels(
    df_labels=df_labels,
    df_geo=df_geo_and_lang,
    col_sort_order='model_sort_order',
    l_ix_subs=['subreddit_id', 'subreddit_name'],
    l_cols_to_front=None,
    geo_cols_to_drop=None,
)

0 <- subs to drop b/c they're not in model
(197, 74) <- df_labels_target.shape
CPU times: user 159 ms, sys: 3.77 ms, total: 163 ms
Wall time: 167 ms


In [15]:
counts_describe(df_labels_target.iloc[:, :15])

,dtype,count,unique,unique-percent,null-count,null-percent
model_sort_order,int64,197,197,100.00%,0,0.00%
subreddit_id,object,197,197,100.00%,0,0.00%
subreddit_name,object,197,197,100.00%,0,0.00%
primary_topic,object,174,39,22.41%,23,11.68%
rating_short,object,191,2,1.05%,6,3.05%
over_18,object,42,1,2.38%,155,78.68%
rating_name,object,191,2,1.05%,6,3.05%
posts_for_modeling_count,int64,197,140,71.07%,0,0.00%
k_0013_label,int64,197,11,5.58%,0,0.00%
k_0023_label,int64,197,19,9.64%,0,0.00%


## Run loop to find "optimal" min_num of subreddits for dynamic clusters


We want to balance two things:
- prevent orphan subreddits
- prevent clusters that are too large to be meaningful

In order to do this at a country level, we'll be better off starting with smallest cluster size and roll up until we have at least N subreddits in one cluster.

Find optimal `min_subreddits_in_cluster` based on:
- `orphan count`, 
- `number of clusters`,
- & other info

number might be different for each country and even within a country it might differ by when we filter NSFW subs.

In [16]:
col_new_cluster_val = 'cluster_label'
col_new_cluster_name = 'cluster_label_k'
col_new_cluster_prim_topic = 'cluster_majority_primary_topic'
col_new_cluster_topic_mix = 'cluster_topic_mix'

### Loop

In [17]:
%%time

df_optimal_min_check, n_min_subs_in_cluster_optimal = get_table_for_optimal_dynamic_cluster_params(
        df_labels_target=df_labels_target,
        col_new_cluster_val=col_new_cluster_val,
        col_new_cluster_name=col_new_cluster_name,
        col_new_cluster_prim_topic=col_new_cluster_prim_topic,
        col_new_cluster_topic_mix=col_new_cluster_topic_mix,
        min_subs_in_cluster_list=np.arange(3, 11),
        verbose=False,
        return_optimal_min_subs_in_cluster=True,
)

100%|██████████| 16/16 [00:00<00:00, 40.54it/s]

100%|██████████| 16/16 [00:00<00:00, 37.83it/s]

100%|██████████| 16/16 [00:00<00:00, 37.51it/s]

100%|██████████| 16/16 [00:00<00:00, 40.20it/s]

100%|██████████| 16/16 [00:00<00:00, 36.34it/s]

100%|██████████| 16/16 [00:00<00:00, 40.67it/s]

100%|██████████| 16/16 [00:00<00:00, 36.78it/s]

100%|██████████| 16/16 [00:00<00:00, 40.17it/s]

100%|██████████| 8/8 [00:06<00:00,  1.22it/s]


CPU times: user 6.35 s, sys: 174 ms, total: 6.53 s
Wall time: 6.63 s


In [18]:
assert (
    n_min_subs_in_cluster_optimal == df_optimal_min_check.loc[
        df_optimal_min_check['num_orphan_subreddits'] == df_optimal_min_check['num_orphan_subreddits'].min(),
        'min_subreddits_in_cluster'
    ].values[0]
)

In [19]:
df_optimal_min_check

,subs_to_cluster_count,min_subreddits_in_cluster,cluster_count,num_orphan_subreddits,num_subreddits_per_cluster_min,num_subreddits_per_cluster_mean,num_subreddits_per_cluster_median,num_subreddits_per_cluster_max,num_clusters_with_mature_primary_topic,cluster_ids_with_orphans
0,197,3,48,4,1,4.104167,4.0,11,7,"0004, 0006, 0012, 0013"
1,197,4,37,3,1,5.324324,5.0,11,6,"0001, 0004, 0006"
2,197,5,31,1,1,6.354839,6.0,12,5,0004
3,197,6,28,2,1,7.035714,7.0,12,3,"0004, 0006"
4,197,7,23,2,1,8.565217,9.0,12,3,"0004, 0013"
5,197,8,21,1,1,9.380952,10.0,17,3,0004
6,197,9,19,2,1,10.368421,11.0,21,3,"0004, 0006"
7,197,10,19,2,1,10.368421,11.0,21,3,"0004, 0006"


### Display loop results

In [20]:
def highlight_below_threshold(val, threshold=1):
    if val <= threshold:
        return "color:purple; font-weight: bold; background-color:yellow;"
    else:
        return ''
print(n_min_subs_in_cluster_optimal)

col_num_orph_subs = 'num_orphan_subreddits'
# col_num_subs_mean = 'num_subreddits_per_cluster_mean'
col_num_subs_median = 'num_subreddits_per_cluster_median'

style_df_numeric(
    df_optimal_min_check,
    rename_cols_for_display=True,
    l_bar_simple=[col_num_orph_subs,
                  col_num_subs_median,]
).applymap(highlight_below_threshold, subset=[col_num_orph_subs.replace('_', ' ')])


5


,subs to cluster count,min subreddits in cluster,cluster count,num orphan subreddits,num subreddits per cluster min,num subreddits per cluster mean,num subreddits per cluster median,num subreddits per cluster max,num clusters with mature primary topic,cluster ids with orphans
,,,,,,,,,,
0,197,3,48,4,1,4.10,4.00,11,7,"0004, 0006, 0012, 0013"
1,197,4,37,3,1,5.32,5.00,11,6,"0001, 0004, 0006"
2,197,5,31,1,1,6.35,6.00,12,5,0004
3,197,6,28,2,1,7.04,7.00,12,3,"0004, 0006"
4,197,7,23,2,1,8.57,9.00,12,3,"0004, 0013"
5,197,8,21,1,1,9.38,10.00,17,3,0004
6,197,9,19,2,1,10.37,11.00,21,3,"0004, 0006"
7,197,10,19,2,1,10.37,11.00,21,3,"0004, 0006"


## Get dyanimc clusters (apply optimal num from above)

side bar: about 57% of subreddits in Australia only had a single primary topic as their `topic_mix`, so combining `primary topic` might not give us as much info as we hoped.

At the same time, for 43% of subs we might get additional detail by combining the primary topics.


In [21]:
print(n_min_subs_in_cluster_optimal)

n_mix_start = 4
l_ix = ['subreddit_id', 'subreddit_name']
col_new_cluster_topic_mix = 'cluster_topic_mix'
col_subreddit_topic_mix = 'subreddit_full_topic_mix'
col_full_depth_mix_count = 'subreddit_full_topic_mix_count'
suffix_new_topic_mix = '_topic_mix_nested'
col_new_cluster_val_int = 'cluster_label_int'

df_labels_target_dynamic_raw = create_dynamic_clusters(
    df_labels_target,
    agg_strategy='aggregate_small_clusters',
    min_subreddits_in_cluster=n_min_subs_in_cluster_optimal,
    l_cols_labels_input=None,
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_prim_topic=col_new_cluster_prim_topic,
    n_mix_start=n_mix_start,
    col_new_cluster_topic_mix=col_new_cluster_topic_mix,
    col_subreddit_topic_mix=col_subreddit_topic_mix,
    col_full_depth_mix_count=col_full_depth_mix_count,
    suffix_new_topic_mix=suffix_new_topic_mix,
    l_ix=l_ix,
    verbose=True,
)

02:08:35 | INFO | "Concat'ing nested cluster labels..."


5


02:08:35 | INFO | "Getting topic mix at different depths..."
02:08:35 | INFO | "  Assigning base topic mix cols"
02:08:35 | INFO | "  Creating deepest base topic mix col..."
02:08:35 | INFO | "  Iterating through additional subs with multiple topics..."
100%|██████████| 16/16 [00:00<00:00, 40.80it/s]
02:08:36 | INFO | "Initializing values for strategy: aggregate_small_clusters"
02:08:36 | INFO | "  Looping to roll-up clusters from smallest to largest..."
100%|██████████| 21/21 [00:00<00:00, 201.07it/s]
02:08:36 | INFO | "(197, 125) <- output shape"


In [22]:
style_df_numeric(
    get_dynamic_cluster_summary(
        df_labels_target_dynamic_raw,
        return_dict=False,
    ),
    rename_cols_for_display=True,
)

,cluster count,num orphan subreddits,num subreddits per cluster min,num subreddits per cluster mean,num subreddits per cluster median,num subreddits per cluster max,num clusters with mature primary topic,cluster ids with orphans
,,,,,,,,
0,31,1,1,6.35,6.00,12,5,0004


In [ ]:
# # # check column order
# style_df_numeric(
#     df_labels_target_dynamic_raw.iloc[104:109, :15],
#     rename_cols_for_display=True,
#     int_labels=['total_users_in', 'num_of_countries_', 'users_in_subreddit_from_country_l28',
#                     'by_country_rank',
#                     ],
#     pct_cols=['b_users_percent_by_subreddit',
#                   'c_users_percent_by_country',
#                   'users_percent_by_country_avg',
#                   ],
#     pct_labels='',
# )

### Minor QA checks

In [23]:
# # check column order
# style_df_numeric(
#     df_labels_target_dynamic_raw.iloc[70:74, -22:],
#     rename_cols_for_display=True,
#     int_labels=['total_users_in', 'num_of_countries_', 'users_in_subreddit_from_country_l28',
#                     'by_country_rank',
#                     ],
#     pct_cols=['b_users_percent_by_subreddit',
#                   'c_users_percent_by_country',
#                   'users_percent_by_country_avg',
#                   ],
#     pct_labels='',
# )

In [24]:
value_counts_and_pcts(
    df_labels_target_dynamic_raw[col_new_cluster_topic_mix],
    top_n=9,
)

,cluster_topic_mix-count,cluster_topic_mix-percent,cluster_topic_mix-pct_cumulative_sum
Mature Themes and Adult Content,24,12.2%,12.2%
Technology,23,11.7%,23.9%
Internet Culture and Memes,18,9.1%,33.0%
Crypto,17,8.6%,41.6%
Gaming,17,8.6%,50.3%
Cars and Motor Vehicles | Sports,15,7.6%,57.9%
Place,15,7.6%,65.5%
Politics,15,7.6%,73.1%
Hobbies,9,4.6%,77.7%


In [25]:
# how many final clusters have multiple topics?
value_counts_and_pcts(
    df_labels_target_dynamic_raw[col_new_cluster_topic_mix].str.count('\|')
)

,cluster_topic_mix-count,cluster_topic_mix-percent,cluster_topic_mix-pct_cumulative_sum
0,176,89.3%,89.3%
1,21,10.7%,100.0%


In [26]:
# how many SUBREDDITS have multiple topics? (when we check the deepest clusters)
#  these two calls are equivalent

# value_counts_and_pcts(
#     df_labels_target_dynamic_raw[col_subreddit_topic_mix].str.count('\|')
# )

value_counts_and_pcts(
    df_labels_target_dynamic_raw[col_full_depth_mix_count]
)

,subreddit_full_topic_mix_count-count,subreddit_full_topic_mix_count-percent,subreddit_full_topic_mix_count-pct_cumulative_sum
1,97,49.2%,49.2%
2,68,34.5%,83.8%
3,20,10.2%,93.9%
4,8,4.1%,98.0%
5,4,2.0%,100.0%


In [27]:
style_df_numeric(
    df_labels_target_dynamic_raw
    [df_labels_target_dynamic_raw[col_full_depth_mix_count] >= 5]
    .iloc[-5:, :9]
    ,
    rename_cols_for_display=True,
)

,subreddit id,subreddit name,cluster label int,cluster topic mix,primary topic,rating short,subreddit full topic mix,rating name,over 18
,,,,,,,,,
98,t5_2qi1n,nederlands,8,Medical and Mental Health,Place,E,"Meta/Reddit | Reading, Writing, and Literature | Funny/Humor | Internet Culture and Memes | Place",Everyone,f
99,t5_2dm6a0,writestreak,8,Medical and Mental Health,Learning and Education,E,"Meta/Reddit | Reading, Writing, and Literature | Funny/Humor | Internet Culture and Memes | Learning and Education",Everyone,-
134,t5_287xxc,notjustbikes,17,Politics,Activism,E,Place | Cars and Motor Vehicles | Travel | Politics | Activism,Everyone,-
196,t5_57m7gn,arjan,13,Animals and Pets,-,-,Music | Mature Themes and Adult Content | Internet Culture and Memes | Meta/Reddit | Learning and Education,-,-


## Re-assign orphan subreddits (optional)

If there are subreddits that are orphan (see summary above), check them out to see if we can re-assign them w/o too much work. if we can't skip and move to the next country.

In [28]:
# # check subs around orphan sub
# # n_plus_minus_ = 5
# ix_orphan_ = (
#     df_labels_target_dynamic_raw
#     [df_labels_target_dynamic_raw[col_new_cluster_val] == '0011']
#     .index
# )
# df_labels_target_dynamic_raw.iloc[ix_orphan_, :9]

In [29]:
# # check other subs that are in the same cluster as orphan sub (at broadest level)
# l_cols_orphan_check = (
#     [
#         'subreddit_id',
#         col_new_cluster_topic_mix, 
#         # col_new_cluster_val,  # this can be really long and makes comparing harder
#         # col_subreddit_topic_mix,
#         'subreddit_name', 
#         col_new_cluster_name
#     ] +
#     l_cols_labels[:-5]
# )

# style_df_numeric(
#     df_labels_target_dynamic_raw
#     [df_labels_target_dynamic_raw['k_0013_label'] == 11]
#     [l_cols_orphan_check]
#     .iloc[3:14, :50]
#     ,
#     l_bar_simple=[c for c in l_cols_orphan_check[4:] if c.endswith('_label')],
#     rename_cols_for_display=True,

# )

In [30]:
# label_k_to_reassign_ = 'k_0320_label'
# label_val_to_reassign_ = '0011-0018-0032-0043-0046-0058-0062-0087-0244'
# subreddit_id_orphan_ = 't5_2tt7r'

# mask_orphan_and_new_group = (
#     (df_labels_target_dynamic_raw['subreddit_id'] == subreddit_id_orphan_) |
#     (
#         (df_labels_target_dynamic_raw[col_new_cluster_name] == label_k_to_reassign_) &
#         (df_labels_target_dynamic_raw[col_new_cluster_val] == label_val_to_reassign_)
#     )
# )

# # assign is similar to what we do in the dynamic function
# label_k_new_ = 'k_0118_label'
# label_val_new_col_ = f"{label_k_new_}_nested"
# new_prim_topic_col_ = label_k_new_.replace('_label', '_majority_primary_topic')
# c_update_topic_mix_ = label_k_new_.replace('_label', suffix_new_topic_mix)

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_name
# ] = label_k_new_

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_val
# ] = df_labels_target_dynamic_raw[mask_orphan_and_new_group][label_val_new_col_]

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_prim_topic
# ] = df_labels_target_dynamic_raw[mask_orphan_and_new_group][new_prim_topic_col_]

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_topic_mix
# ] = df_labels_target_dynamic_raw[mask_orphan_and_new_group][c_update_topic_mix_]

# del mask_orphan_and_new_group, label_k_to_reassign_, label_val_to_reassign_
# del label_k_new_, label_val_new_col_, new_prim_topic_col_

In [31]:
# # check again, num of orphans should be lower than before
# style_df_numeric(
#     get_dynamic_cluster_summary(
#         df_labels_target_dynamic_raw,
#         return_dict=False,
#     ),
#     rename_cols_for_display=True,
# )

In [32]:
# value_counts_and_pcts(
#     df_labels_target_dynamic_raw,
#     ['cluster_label'],
#     top_n=None,
#     return_df=True
# )['count'].describe()

## Get cluster for humans (list of subs in a cluster in a cell)
Here we get 1 cluster per row. 
Use cases:
- It makes it easier to quickly check NSFW clusters that we'll filter out
- we'll append the list of subreddit names from here to the final table for QA (makes it easier to evaluate whether the cluster makes sense).


In [33]:
col_subs_in_cluster_count = 'subs_in_cluster_count'
col_list_cluster_names = 'list_cluster_subreddit_names'

df_cluster_for_humans = reshape_df_to_get_1_cluster_per_row(
    df_labels_target_dynamic_raw,
    col_counterpart_count=col_subs_in_cluster_count,
    col_list_cluster_names=col_list_cluster_names,
    col_list_cluster_ids='list_cluster_subreddit_ids',
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_val_int=col_new_cluster_val_int,
    col_new_cluster_topic=col_new_cluster_topic_mix,
    verbose=False,
    get_one_column_per_sub_id=False,
)

(31, 7)


In [34]:
df_cluster_for_humans.iloc[10:15, :-1] # .iloc[40:45, :]

,cluster_label,cluster_label_k,cluster_topic_mix,cluster_label_int,subs_in_cluster_count,list_cluster_subreddit_names
10,0006-0010-0015-0021-0021-0028-0029,k_0085_label,Technology,29,6,"klussers, fanatec, hue, photogrammetry, homeassistant, eufycam"
11,0006-0010-0017-0023-0023-0031-0032-0040-0115-0227-0331-0377,k_1065_label,Gaming,377,6,"citiesskylines, shittyskylines, openrct2, rct, themeparkitect, planetcoaster"
12,0007,k_0013_label,Place,7,4,"suriname, expats, aruba, curacao"
13,0007-0011-0019-0026-0026-0035-0036-0046-0132-0265-0393-0443-0642-0757-0911-0969-1172-1302-1402-1529-1597-1622,k_3927_label,Place,1622,11,"maastricht, eindhoven, utrecht, enschede, groningen, nijmegen, thehague, rotterdam, amsterdam, netherlands, antwerpen"
14,0007-0012,k_0023_label,Learning and Education,12,6,"studyinthenetherlands, tudelft, dutchfire, geldzaken, beleggen, juridischadvies"


In [ ]:
# df_cluster_for_humans.tail(9)

### Check clusters that have mature topics

Make a list of cluster IDs to exclude for clean df

In [35]:
mask_mature_clusters_ = (
    df_cluster_for_humans[col_new_cluster_topic_mix].str.lower()
    .str.contains('mature')
)
print(f"{mask_mature_clusters_.sum()} <- clusters with 'Mature' in topic mix")

5 <- clusters with 'Mature' in topic mix


In [36]:
# check the first few clusters
(
    df_cluster_for_humans
    [mask_mature_clusters_]
    .iloc[:5, :-1]
)

,cluster_label,cluster_label_k,cluster_topic_mix,cluster_label_int,subs_in_cluster_count,list_cluster_subreddit_names
0,0001,k_0013_label,Mature Themes and Adult Content,1,5,"deevanderzeeuw, quceehumor, lieszhara, lieszharax, nelphelanfan"
1,0001-0001-0003-0003-0003-0004,k_0079_label,Mature Themes and Adult Content,4,6,"lingerielover, eotbdutch, exonthebeachtempafap, franciskweldam, jade_fap, chanelphelan"
2,0002-0002-0005-0005-0005,k_0063_label,Mature Themes and Adult Content,5,6,"myronkoops_, myron_koops_sexy, myronkoops, jameciabakerfapnew, rosannavoorwald2, famkelouisefap"
3,0004,k_0013_label,Mature Themes and Adult Content,4,1,euro4euro
16,0008-0014-0025-0034-0036,k_0063_label,Mature Themes and Adult Content,36,6,"researchchemicalsnl, 2cb, researchchemicals, untappd, drinkinggames, amsterdaments"


In [37]:
# (
#     df_cluster_for_humans
#     [mask_mature_clusters_]
#     .iloc[-12:, :]
# )

In [46]:
# might need to re-import in order for updates to show up
from subclu.models.reshape_clusters_v041 import _L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_, flag_mature_clusters_to_exclude_from_qa

print(len(_L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_))
_L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_[25:35]

72


['0001-0001-0003-0003-0003-0004-0004-0004-0010-0019',
 '0001-0001-0003-0003-0003-0004-0005-0005',
 '0001-0001-0001-0001-0001-0002-0002-0002-0005-0009-0012-0014',
 '0001-0001-0002-0002-0002-0003-0003-0003-0009-0017',
 '0001-0001-0003-0003-0003-0004-0005-0005-0011-0020-0029',
 '0001-0001-0003-0003-0003-0004-0005-0005-0011-0020-0030',
 '0001-0001-0003-0003-0003-0004-0005-0005-0012',
 '0001-0001-0003-0003-0003-0004-0005-0005-0012-0022-0032',
 '0001-0001-0003-0003-0003-0004-0005-0005-0012-0023-0033',
 '0001-0001-0003-0003-0003-0004-0005-0005-0013']

In [47]:
# sorted(_L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_)

### Add the flag to exclude from QA & the list of sub names to df-raw

In [48]:
val_exclude_from_qa = 'exclude from QA'
col_exclude_from_qa = 'exclude_from_qa'


df_cluster_for_humans[col_exclude_from_qa] = flag_mature_clusters_to_exclude_from_qa(
    df_cluster_list=df_cluster_for_humans,
    col_new_cluster_val=col_new_cluster_val,
    col_exclude_from_qa=col_exclude_from_qa,
    val_exclude_from_qa=val_exclude_from_qa,
    list_known_nsfw_labels=_L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_,
    depth_to_exclude=10,
    verbose=True,
)
print(f"{mask_mature_clusters_.sum()} <- clusters with 'Mature' in topic mix")
display(value_counts_and_pcts(df_cluster_for_humans[col_exclude_from_qa]))


df_labels_target_dynamic_raw = (
    df_labels_target_dynamic_raw
    # drop in case we're re-assigning the values
    .drop(
        [c for c in [col_list_cluster_names, col_subs_in_cluster_count, col_exclude_from_qa] if c in df_labels_target_dynamic_raw.columns],
        axis=1
    )
    .merge(
        df_cluster_for_humans[[col_new_cluster_val, col_list_cluster_names, col_subs_in_cluster_count, col_exclude_from_qa]],
        how='left',
        on=[col_new_cluster_val],
    )
)

df_labels_target_dynamic_raw.shape

5 <- clusters with 'Mature' in topic mix


,exclude_from_qa-count,exclude_from_qa-percent,exclude_from_qa-pct_cumulative_sum
keep,27,87.1%,87.1%
exclude from QA,4,12.9%,100.0%


(197, 128)

In [49]:
mask_mature_clusters_ = (
    df_cluster_for_humans[col_new_cluster_topic_mix].str.lower()
    .str.contains('mature')
)
print(f"{mask_mature_clusters_.sum()} <- clusters with 'Mature' in topic mix")

mask_keep_clusters = (
    df_cluster_for_humans[col_exclude_from_qa] != val_exclude_from_qa
)

print(f"\nCluster IDs that are mature but kept:")
# get list of cluster-IDs to add to exclude to mature list
(
    df_cluster_for_humans
    [mask_mature_clusters_ & mask_keep_clusters]
    ['cluster_label']
    .values
    [:15]
)

5 <- clusters with 'Mature' in topic mix

Cluster IDs that are mature but kept:


array(['0008-0014-0025-0034-0036'], dtype=object)

In [50]:
(
    df_cluster_for_humans
    [mask_mature_clusters_ & mask_keep_clusters]
    .iloc[:5, :]
)

,cluster_label,cluster_label_k,cluster_topic_mix,cluster_label_int,subs_in_cluster_count,list_cluster_subreddit_names,list_cluster_subreddit_ids,exclude_from_qa
16,0008-0014-0025-0034-0036,k_0063_label,Mature Themes and Adult Content,36,6,"researchchemicalsnl, 2cb, researchchemicals, untappd, drinkinggames, amsterdaments","t5_2hcwfn, t5_2xazx, t5_2rhqm, t5_2vnul, t5_2qmza, t5_2vnvc",keep


## Create new df_clean 

- Add list of subreddits to target-CLEAN, b/c we'll need it for rating final
- Add new columns & update order


### Copy baseline cols for clean

In [51]:
col_model_sort_order = 'model_sort_order'

df_labels_target_dynamic_raw, df_labels_target_dynamic_clean = create_dynamic_clusters_clean(
    df_dynamic_raw=df_labels_target_dynamic_raw,
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_prim_topic=col_new_cluster_prim_topic,
    col_new_cluster_topic_mix=col_new_cluster_topic_mix,
    col_subreddit_topic_mix=col_subreddit_topic_mix,
)

# check num of orphans for clean
display(style_df_numeric(
    get_dynamic_cluster_summary(
        df_labels_target_dynamic_clean.rename(columns={c: c.replace(' ', '_') for c in df_labels_target_dynamic_clean.columns}),
        return_dict=False,
    ),
    rename_cols_for_display=True,
).set_caption(f"Summary for df-CLEAN"))


# print(f"{len(l_cols_clean_final_for_qa)} <- expected final col count")
print(f"{df_labels_target_dynamic_raw.shape} <- df raw shape")
print(f"{df_labels_target_dynamic_clean.shape} <- df clean shape")

,cluster count,num orphan subreddits,num subreddits per cluster min,num subreddits per cluster mean,num subreddits per cluster median,num subreddits per cluster max,num clusters with mature primary topic,cluster ids with orphans
,,,,,,,,
0,27,0,2,6.63,6.00,12,1,


(197, 130) <- df raw shape
(179, 33) <- df clean shape


In [52]:
# [c for c in df_labels_target_dynamic_clean.columns if 'cluster' in c]

In [53]:
# df_labels_target_dynamic_clean.iloc[:5, :29]

In [54]:
# df_labels_target_dynamic_clean.iloc[:5, -10:]

## Create target to target list for FPR example

In [55]:
%%time
print(datetime.utcnow())

df_target_to_target_list = convert_distance_or_ab_to_list_for_fpr(
    df_labels_target_dynamic_clean.rename(columns={c: c.replace(' ', '_') for c in df_labels_target_dynamic_clean.columns}),
    convert_to_ab=True,
    col_counterpart_count='counterpart_count',
    col_list_cluster_names='list_cluster_subreddit_names',
    col_list_cluster_ids='list_cluster_subreddit_ids',
    l_cols_for_seeds=None,
    l_cols_for_clusters=None,
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_prim_topic=col_new_cluster_prim_topic,
    verbose=False,
)
df_target_to_target_list.shape

2022-04-07 02:14:00.743182
  (1146, 9) <- df_ab.shape after removing matches to self
  (160, 9) <- df_a_to_b.shape
CPU times: user 32.3 ms, sys: 54 µs, total: 32.4 ms
Wall time: 31.4 ms


In [56]:
df_target_to_target_list.drop('list_cluster_subreddit_ids', axis=1).iloc[:5, :11]

,subreddit_id_seed,subreddit_name_seed,cluster_label,cluster_label_k,primary_topic,cluster_majority_primary_topic,counterpart_count,list_cluster_subreddit_names
0,t5_331vx,ti84hacks,0006,k_0013_label,Technology,Technology,4,"super73, vanmoofbicycle, cowboybikes, bikerace"
1,t5_45ae4t,tplink_omada,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,Technology,Technology,11,"googlehome, pihole, intune, usenet, radarr, unifi, protonvpn, lastpass, f1tv, logitechharmony, heos"
2,t5_2rfb1,unifi,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,Technology,Technology,11,"googlehome, pihole, intune, usenet, radarr, protonvpn, lastpass, f1tv, logitechharmony, tplink_omada, heos"
3,t5_3i1z1,protonvpn,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,Technology,Technology,11,"googlehome, pihole, intune, usenet, radarr, unifi, lastpass, f1tv, logitechharmony, tplink_omada, heos"
4,t5_39spl,pihole,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,Technology,Technology,11,"googlehome, intune, usenet, radarr, unifi, protonvpn, lastpass, f1tv, logitechharmony, tplink_omada, heos"


# Export data

Note that we have to use `fillna('')`

Otherwise, we'll get errors because the gspread library doesn't know how to handle `pd.NaN` or `np.Nan` (nulls).

## Define variables to create/access google sheet doc & worksheets

Moved the defintion to the top of the sheet so it's easier to automate.

## Save: Clean sheet to rate

In [57]:
d_wsh_names.keys()

dict_keys(['qa_ready', 'clusters_t2t_list_raw', 'clusters_t2t_fpr_after_qa', 'sub_raw'])

In [58]:
%%time
print(datetime.utcnow())
(
    d_wsh_names['qa_ready']['worksheet']
    .update([df_labels_target_dynamic_clean.columns.values.tolist()] + 
             df_labels_target_dynamic_clean.fillna('').values.tolist())
)

2022-04-07 02:14:24.553449
CPU times: user 34.9 ms, sys: 0 ns, total: 34.9 ms
Wall time: 363 ms


## Save: df cluster for humans

In [59]:
%%time
print(datetime.utcnow())
(
    d_wsh_names['clusters_t2t_list_raw']['worksheet']
    .update(
        [df_cluster_for_humans.rename(columns={c: c.replace('_', ' ') for c in df_cluster_for_humans}).columns.values.tolist()] + 
        df_cluster_for_humans.fillna('').values.tolist()
    )
)

2022-04-07 02:14:25.491047
CPU times: user 7.42 ms, sys: 24 µs, total: 7.44 ms
Wall time: 128 ms


## Save: target raw dynamic


In [60]:
l_cols_to_drop = (
    ['table_creation_date'] +
    [c for c in df_labels_target_dynamic_raw.columns if c.endswith('_nested')]
)
print(len(l_cols_to_drop))
# df_labels_target_dynamic_raw.columns.to_list()

45


In [61]:
%%time
print(datetime.utcnow())
(
    d_wsh_names['sub_raw']['worksheet']
    .update([df_labels_target_dynamic_raw.drop(l_cols_to_drop, axis=1).columns.values.tolist()] + 
             df_labels_target_dynamic_raw.drop(l_cols_to_drop, axis=1).fillna('').values.tolist())
)

2022-04-07 02:14:26.757394
CPU times: user 18.5 ms, sys: 1.86 ms, total: 20.4 ms
Wall time: 300 ms


## Save: FPR target-2-target as list

Even though data isn't fully ready, want to have the output ready to make sure it's in the right format that we need.

UPDATE: stop exporting this for now because it adds noise and could create confusion between it and the final QA sheet.

In [62]:
df_target_to_target_list.iloc[:5, :5]

,subreddit_id_seed,subreddit_name_seed,cluster_label,cluster_label_k,primary_topic
0,t5_331vx,ti84hacks,0006,k_0013_label,Technology
1,t5_45ae4t,tplink_omada,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,Technology
2,t5_2rfb1,unifi,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,Technology
3,t5_3i1z1,protonvpn,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,Technology
4,t5_39spl,pihole,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,Technology


In [63]:
# %%time
# print(datetime.utcnow())
# (
#     d_wsh_names['clusters_t2t_fpr_raw']['worksheet']
#     .update(
#         [df_target_to_target_list.columns.values.tolist()] + 
#         df_target_to_target_list.fillna('').values.tolist()
#     )
# )

### We can read the data back to confirm it's as expected

In [64]:
# Here's how to get the records as a dataframe
pd.DataFrame(
    d_wsh_names['qa_ready']['worksheet'].get_all_records()
).iloc[:5, :15]

,subreddit id,subreddit name,cluster label int,cluster topic mix,not country relevant,rated E,relevant to cluster/ other subreddits in cluster,safe to show in relation to cluster,country relevance notes,rating or cluster notes,link to sub,subs in cluster count,list cluster subreddit names,posts for modeling count,users l7
0,t5_3fbpt,super73,6,Technology,,TRUE,,,,,www.reddit.com/r/super73,5,"ti84hacks, super73, cowboybikes, vanmoofbicycle, bikerace",516,5549
1,t5_vwvbb,vanmoofbicycle,6,Technology,,TRUE,,,,,www.reddit.com/r/vanmoofbicycle,5,"ti84hacks, super73, cowboybikes, vanmoofbicycle, bikerace",293,4151
2,t5_331vx,ti84hacks,6,Technology,,TRUE,,,,,www.reddit.com/r/ti84hacks,5,"ti84hacks, super73, cowboybikes, vanmoofbicycle, bikerace",64,1534
3,t5_10wzrv,cowboybikes,6,Technology,,TRUE,,,,,www.reddit.com/r/cowboybikes,5,"ti84hacks, super73, cowboybikes, vanmoofbicycle, bikerace",90,1056
4,t5_2vbm9,bikerace,6,Technology,,TRUE,,,,,www.reddit.com/r/bikerace,5,"ti84hacks, super73, cowboybikes, vanmoofbicycle, bikerace",211,584


# Appendix


## Additional checks on cluster depth

In [70]:
print(df_labels_target_dynamic_raw['cluster_label'].nunique())
display(
    value_counts_and_pcts(
        df_labels_target_dynamic_raw,
        ['cluster_label'],
        top_n=10,
        count_type='subreddit',
        rename_cols_for_display=True,
    )
)
value_counts_and_pcts(
    df_labels_target_dynamic_raw,
    ['cluster_label'],
    top_n=None,
    return_df=True,
)['count'].describe()

31


,subreddit count,percent of subreddit,cumulative percent of subreddit
cluster label,,,
0006-0008-0013-0017-0017-0021-0022-0026,12,6.1%,6.1%
0007-0011-0019-0026-0026-0035-0036-0046-0132-0265-0393-0443-0642-0757-0911-0969-1172-1302-1402-1529-1597-1622,11,5.6%,11.7%
0013-0022-0038-0055-0058-0073-0078-0108,10,5.1%,16.8%
0006-0010,9,4.6%,21.3%
0008,8,4.1%,25.4%
0013-0022-0038-0055-0058-0074-0079,8,4.1%,29.4%
0006-0009-0014-0018-0018-0025-0026-0031-0089-0174-0258-0295,8,4.1%,33.5%
0010-0017-0030-0040-0043-0054-0057-0081-0226-0448-0647-0723-1045-1229,8,4.1%,37.6%
0009-0015-0028-0038-0040-0051-0054,8,4.1%,41.6%


count    31.000000
mean      6.354839
std       2.388143
min       1.000000
25%       5.000000
50%       6.000000
75%       8.000000
max      12.000000
Name: count, dtype: float64

### How deep are the clusters?



In [69]:
print(df_labels_target_dynamic_raw[col_new_cluster_name].nunique())
value_counts_and_pcts(
    df_labels_target_dynamic_raw,
    [col_new_cluster_name],
    top_n=None,
    sort_index=True,
    count_type='subreddit',
    rename_cols_for_display=True,
)

12


,subreddit count,percent of subreddit,cumulative percent of subreddit
cluster label k,,,
k_0013_label,41,20.8%,20.8%
k_0023_label,22,11.2%,32.0%
k_0041_label,6,3.0%,35.0%
k_0063_label,19,9.6%,44.7%
k_0079_label,6,3.0%,47.7%
k_0085_label,22,11.2%,58.9%
k_0118_label,28,14.2%,73.1%
k_0320_label,6,3.0%,76.1%
k_1065_label,14,7.1%,83.2%


In [ ]:
# style_df_numeric(
#     df_labels_target.tail(10),
#     # rename_cols_for_display=True,
#     l_bar_simple=[c for c in df_labels_target.columns if '_label' in c]
# )